### Implementation of the CASE algorithm proposed in the paper "Ensuring Fairness under Prior Probability Shifts"

    - Find the paper here: https://dl.acm.org/doi/pdf/10.1145/3461702.3462596 (Main Reference)
    - ArXiv version of the paper: https://arxiv.org/pdf/2005.03474.pdf
   

1. Reproducing the results shown in Table-1 of the main reference (on synthetic dataset described in the paper)
          
2. Reproducing the results shown in Table-3 of the main reference for Pre-processing techniques. This includes comparing CAPE (CAPE-D and CAPE-1) against Pre (Reweigh) for all the metrics shown in the Table-3, for both COMPASS and MEPS datasets.
                   
3. Reproducing the results shown in Table-3 of the main reference for In-processing techniques. This includes comparing CAPE (CAPE-D and CAPE-1) against one set of In-processing technique for all the metrics shown in the Table-3, for both COMPASS and MEPS datasets.

In [1]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from random import choices
from sklearn.ensemble import GradientBoostingClassifier
import datetime as dt
import pandas as pd
import numpy as np
from sklearn.naive_bayes import GaussianNB
import numpy as np
import math
from sklearn.linear_model import LogisticRegression

## Synthetic Dataset

In [2]:
pv_values = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

In [3]:
def generate_synthetic_dataset(P_1=0.5, P_0=0.5, L=50000):
    a0 = int(P_0*L/2)
    a1 = int(P_1*L/2)
    b0 = int((1-P_0)*L/2)
    b1 = int((1-P_1)*L/2)
    Y = a0*[1] + a1*[1] + b0*[0] + b1*[0]
    U = np.concatenate([np.random.normal(15, 10, a0), np.random.normal(5, 5, a1), np.random.normal(15, 10, b0), np.random.normal(5, 5, b1)])
    V = np.concatenate([np.random.normal(20, 10, a0), np.random.normal(40, 10, a1), np.random.normal(20, 10, b0), np.random.normal(40, 10, b1)])
    Z = a0*[1] + a1*[0] + b0*[1] + b1*[0]
    
    dataset = pd.DataFrame({'U': U, 'V': V, 'Z': Z, 'Y': Y})
    Y_df = dataset.pop('Y')
    
    return dataset, Y_df

In [4]:
syn_train_X, syn_train_Y = generate_synthetic_dataset()

In [5]:
syn_test_X_1, syn_test_Y_1 = generate_synthetic_dataset(0.1, 0.1)

In [11]:
s_t = CASE(syn_train_X, syn_test_X_1, syn_train_Y, syn_test_Y_1)
s_t.partition_z()
print("Generating quantifiers...")
s_t.create_quantifiers()
print("Generating classifiers...")
s_t.create_classifiers(ds='syn')
print("Getting prediction prevalance...")
best_1, best_0 = s_t.get_prediction_prevelance()

Generating quantifiers...
18750
18750
Generating classifiers...
Z: 0  | theta: 0.05
Z: 0  | theta: 0.25
Z: 0  | theta: 0.35
Z: 0  | theta: 0.45
Z: 0  | theta: 0.15
Z: 0  | theta: 0.55
Z: 0  | theta: 0.65
Z: 0  | theta: 0.75
Z: 0  | theta: 0.85
Z: 0  | theta: 0.95
Z: 1  | theta: 0.05
Z: 1  | theta: 0.25
Z: 1  | theta: 0.35
Z: 1  | theta: 0.45
Z: 1  | theta: 0.15
Z: 1  | theta: 0.55
Z: 1  | theta: 0.65
Z: 1  | theta: 0.75
Z: 1  | theta: 0.85
Z: 1  | theta: 0.95
Getting prediction prevalance...
Z: 0  | theta: 0.05
LogisticRegression()
s 0.0
Z: 0  | theta: 0.25
LogisticRegression()
s 0.0
Z: 0  | theta: 0.35
LogisticRegression()
s 0.0
Z: 0  | theta: 0.45
LogisticRegression()
s 0.0
Z: 0  | theta: 0.15
LogisticRegression()
s 0.0
Z: 0  | theta: 0.55
LogisticRegression()
s 1.0
Z: 0  | theta: 0.65
LogisticRegression()
s 1.0
Z: 0  | theta: 0.75
LogisticRegression()
s 1.0
Z: 0  | theta: 0.85
LogisticRegression()
s 1.0
Z: 0  | theta: 0.95
LogisticRegression()
s 1.0
Z: 1  | theta: 0.05
LogisticRegre

([LogisticRegression(), 0.09375999999963192, 1.0],
 [LogisticRegression(), 0.32687999999998574, 0.0])

In [32]:
def generate_split_10(pp, pn):
    X, Y = generate_synthetic_dataset(pp, pn)
    combined = pd.concat([X, Y], axis=1)
    Z0 = combined[combined["Y"] == 0]
    Z0.reset_index()
    Z0_Y = Z0.pop("Y")
    
    Z1 = combined[combined["Y"] == 1]
    Z1.reset_index()
    Z1_Y = Z1.pop("Y")
    
    return Z0, Z0_Y, Z1, Z1_Y

In [ ]:
# generating results for diff. types of synthetic datasets

p_valuesss = [[0.1, 0.1], [0.8, 0.2], [0.9, 0.9]]

for i in p_valuesss:
    Z0, Z0_Y, Z1, Z1_Y = generate_split_10(i[0], i[1])
    s_t = CASE(syn_train_X, syn_test_X_1, syn_train_Y, syn_test_Y_1)
    s_t.partition_z()
    print("Generating quantifiers...")
    s_t.create_quantifiers()
    print("Generating classifiers...")
    s_t.create_classifiers(ds='syn')
    print("Getting prediction prevalance...")
    best_1, best_0 = s_t.get_prediction_prevelance()
    print("----REPORT-----")
    print("For z = 0, prevelance =", i[0])
    print(best_0[0].score(st_1_Z0, st_1_Z0_Y))
    print("--")
    print("For z = 1, prevelance =", i[1])
    print(best_1[0].score(st_1_Z1, st_1_Z1_Y))

## Preprocessing

### 1. COMPAS dataset

In [37]:
def preprocess_compas(df, test_size=0.2):
    # remove empty c_charge_desc columns
    df = df.copy()
    df = df[['race', 'c_jail_in', 'c_jail_out', 'decile_score', 'juv_fel_count', 'juv_misd_count', 'juv_other_count', 'priors_count', 'days_b_screening_arrest', 'c_charge_degree', 'is_recid', 'r_charge_degree', 'is_violent_recid', 'score_text', 'two_year_recid']]
    to_date_columns = ['c_jail_in', 'c_jail_out']
    to_int_columns = ['juv_fel_count', 'decile_score', 'juv_misd_count', 'juv_other_count', 'priors_count', 'days_b_screening_arrest', 'is_recid', 'is_violent_recid', 'two_year_recid']
    
    for col in to_date_columns:
        df[col] = pd.to_datetime(df[col]).dt.date
        
    for col in to_int_columns:
        df[col] = pd.to_numeric(df[col])
        
    df["length_of_stay"] = (df['c_jail_out'] - df['c_jail_in']).dt.days
    df['length_of_stay'].fillna(0, inplace=True)

    
    df = df[df['race'].isin(['Caucasian', 'African-American'])]
    
    df.drop(['c_jail_in', 'c_jail_out', 'r_charge_degree', 'days_b_screening_arrest', 'score_text'], axis=1, inplace=True)

    
    df = pd.get_dummies(df, columns = ['race', 'c_charge_degree'])
    
    df_Y = df.pop('two_year_recid')
    df = df.drop(['race_Caucasian', 'c_charge_degree_M'], axis=1)
    df = df.rename(columns = {'race_African-American': 'Z', 'c_charge_degree_F': 'Felony'})
    print("Shape:", df.shape)
    
    
    return df, df_Y

## Datasets

### COMPAS dataset
Link: https://github.com/propublica/compas-analysis/blob/master/compas-scores-two-years.csv

In [38]:
compas_df = pd.read_csv("data/compas-scores-two-years.csv")

In [39]:
compas_df['screening_date'] = pd.to_datetime(compas_df['screening_date'])
compas_df_2013 = compas_df[compas_df['screening_date'].dt.year == 2013]
compas_df_2014 = compas_df[compas_df['screening_date'].dt.year == 2014]

In [40]:
X_train_2013, Y_train_2013 = preprocess_compas(compas_df_2013)
X_test_2014, Y_test_2014 = preprocess_compas(compas_df_2014)

Shape: (4335, 10)
Shape: (1815, 10)


In [42]:
X_train_2013.reset_index(drop=True, inplace=True)
X_test_2014.reset_index(drop=True, inplace=True)
Y_train_2013.reset_index(drop=True, inplace=True)
Y_test_2014.reset_index(drop=True, inplace=True)

In [43]:
compas_df_2013 = pd.concat([X_train_2013, Y_train_2013], axis=1)
compas_df_2014 = pd.concat([X_test_2014, Y_test_2014], axis=1)

In [44]:
X_train_2013

,decile_score,juv_fel_count,juv_misd_count,juv_other_count,priors_count,is_recid,is_violent_recid,length_of_stay,Z,Felony
0,3,0,0,0,0,1,1,10.0,1,1
1,4,0,0,1,4,1,0,1.0,1,1
2,8,0,1,0,1,0,0,0.0,1,1
3,3,0,0,0,1,1,1,1.0,0,1
4,4,0,0,0,0,0,0,1.0,0,1
...,...,...,...,...,...,...,...,...,...,...
4330,4,0,0,0,2,0,0,6.0,1,0
4331,10,0,2,1,5,1,1,13.0,0,0
4332,6,0,0,0,0,1,0,3.0,0,0
4333,9,0,0,0,0,0,0,1.0,1,1


In [46]:
Y_test_2014

0       1
1       0
2       1
3       1
4       1
       ..
1810    0
1811    1
1812    1
1813    0
1814    0
Name: two_year_recid, Length: 1815, dtype: int64

In [45]:
X_test_2014

,decile_score,juv_fel_count,juv_misd_count,juv_other_count,priors_count,is_recid,is_violent_recid,length_of_stay,Z,Felony
0,6,0,0,0,14,1,0,6.0,0,1
1,1,0,0,0,0,0,0,3.0,0,0
2,3,0,0,0,7,1,0,3.0,1,1
3,3,0,0,0,5,1,0,1.0,0,1
4,6,0,0,0,13,1,0,1.0,1,1
...,...,...,...,...,...,...,...,...,...,...
1810,2,0,0,0,1,0,0,1.0,1,1
1811,10,0,1,0,19,1,0,9.0,1,1
1812,2,0,0,0,0,1,0,1.0,1,0
1813,3,0,0,0,0,0,0,2.0,1,1


### Scaled Probability Average Quantifier
_A. Bella, C. Ferri, J. Hernández-Orallo and M. J. Ramírez-Quintana, "Quantification via Probability Estimators," 2010 IEEE International Conference on Data Mining, 2010, pp. 737-742, doi: 10.1109/ICDM.2010.75._

## Main pipeline

In [34]:
class CASE():
    def __init__(self, X_train, X_test, Y_train, Y_test):
        self.X_train = X_train
        self.X_test = X_test
        self.Y_train = Y_train
        self.Y_test = Y_test
        self.THETA = {0.05, 0.15, 0.25, 0.35, 0.45, 0.55, 0.65, 0.75, 0.85, 0.95}
        self.Z = ["0", "1"]
        self.all_classifiers = {"0": [], "1": []}
        self.all_quantifiers = []
        
        
    # TRAINING PHASE
    
    # step 1
    def partition_z(self):
        X_t_Z_1_arr = []
        X_t_Z_0_arr = []
        X_te_Z_1_arr = []
        X_te_Z_0_arr = []
        Y_t_Z_1_arr = []
        Y_t_Z_0_arr = []
        Y_te_Z_1_arr = []
        Y_te_Z_0_arr = []
        for idx, row in self.X_train.iterrows():
            if row['Z']:
                X_t_Z_1_arr.append(self.X_train.iloc[[idx]])
                Y_t_Z_1_arr.append(self.Y_train.iloc[[idx]])
            else:
                X_t_Z_0_arr.append(self.X_train.iloc[[idx]])
                Y_t_Z_0_arr.append(self.Y_train.iloc[[idx]])
                
        for idx, row in self.X_test.iterrows():
            if row['Z']:
                X_te_Z_1_arr.append(self.X_test.iloc[[idx]])
                Y_te_Z_1_arr.append(self.Y_test.iloc[[idx]])
            else:
                X_te_Z_0_arr.append(self.X_test.iloc[[idx]])
                Y_te_Z_0_arr.append(self.Y_test.iloc[[idx]])
                
        all_arrs = [X_t_Z_1_arr, X_te_Z_1_arr, Y_t_Z_1_arr, Y_te_Z_1_arr, X_t_Z_0_arr, X_te_Z_0_arr, Y_t_Z_0_arr, Y_te_Z_0_arr]
        
        for idx, arr in enumerate(all_arrs):
            all_arrs[idx] = pd.concat(arr)
                
                
        self.D_z = {
            "0": all_arrs[4:],
            "1": all_arrs[:4]
        }
        
        return 0
    
    # scaled probability algorithm
    def spa(self, pred, pred_probs):
        total = pred.shape[0]
        num_pos = sum(pred)
        num_neg = total - num_pos

        p_sum_prob = 0
        n_sum_prob = 0

        for idx, val in enumerate(pred):
            if val:
                p_sum_prob += pred_probs[idx][1]
            else:
                n_sum_prob += pred_probs[idx][1]
        p_sum_prob /= num_pos
        n_sum_prob /= num_neg

        pa = (sum(i[1] for i in pred_probs))/total
        spa = (pa - n_sum_prob)/(p_sum_prob - n_sum_prob)

        return spa
        
    # step 2
    def create_quantifiers(self, test_size = 0.25):
        for z in self.Z:
            X_train, Y_train = self.D_z[z][0], self.D_z[z][2]
            X_train, X_test, Y_train, Y_test = train_test_split(X_train, Y_train, test_size=test_size)
            print(len(X_train))
            naive_classifier = GaussianNB()
            naive_classifier.fit(X_train, Y_train)
            pred = naive_classifier.predict(X_test)
            pred_probs = naive_classifier.predict_proba(X_test)
            this_spa = self.spa(pred, pred_probs)
            self.all_quantifiers.append(this_spa)
        
        return 0
    
    
    def pp_sampling(self, prevelance, X, Y, ds='COMPAS'):
        df = pd.concat([X, Y], axis=1)
        tot = X.shape[0]

        ds_col_name = {
            'COMPAS': 'two_year_recid',
            'syn': 'Y'
        }
        col_name = ds_col_name[ds]        
        df_0 = df[(df[col_name] == '0') | (df[col_name] == 0)]
        df_1 = df[(df[col_name] == '1') | (df[col_name] == 1)]
        
        num1 = int(prevelance*tot)
        num0 = tot - num1
        
        df_0_final = df_0.iloc[np.random.randint(0, len(df_0), size=num0)]
        df_1_final = df_1.iloc[np.random.randint(0, len(df_1), size=num1)]


        shuffled_df = pd.concat([df_0_final, df_1_final]).sample(frac=1)
        new_Y = shuffled_df.pop(col_name)
        return shuffled_df, new_Y
    
    def create_classifiers(self, ds):
        for z in self.Z:
            for theta in self.THETA:
#                 print("Z:", z, " | theta:", theta)
                new_X_train, new_Y_train = self.pp_sampling(theta, self.X_train, self.Y_train, ds=ds)
                if ds == "syn":
#                     print(new_X_train.describe(), new_Y_train.describe())
                    clf = LogisticRegression().fit(new_X_train, new_Y_train)
                else:
                    clf = GradientBoostingClassifier().fit(new_X_train, new_Y_train)
                self.all_classifiers[z].append(clf)
                
        return 0
    
    # PREDICTION PHASE
    def get_prediction_prevelance(self):
        classifier_pred = {
            "0": [],
            "1": []
        }
        for z in self.Z:
            for idx, theta in enumerate(self.THETA):
#                 print("Z:", z, " | theta:", theta)
                this_clf = self.all_classifiers[z][idx]
#                 print(this_clf)
                this_pred = this_clf.predict(self.D_z[z][1])
#                 print("s", sum(this_pred)/len(this_pred))
                classifier_pred[z].append([this_clf, abs((sum(this_pred)/len(this_pred)) - self.all_quantifiers[int(z)]), (sum(this_pred)/len(this_pred))])
                
        zero_clfs = classifier_pred["0"]
        one_clfs = classifier_pred["1"]
        
        zero_clfs.sort(key=lambda x:x[1])
        one_clfs.sort(key=lambda x:x[1])
        
        best_0 = zero_clfs[0]
        best_1 = one_clfs[0]
        
        return best_0, best_1

In [67]:
training_stuff = CASE(X_train_2013, X_test_2014, Y_train_2013, Y_test_2014)
training_stuff.partition_z()
print("Generating quantifiers...")
training_stuff.create_quantifiers()
print("Generating classifiers...")
training_stuff.create_classifiers('COMPAS')
print("Getting prediction prevalance...")
training_stuff.get_prediction_prevelance()

Generating quantifiers...
1307
1944
Generating classifiers...
Getting prediction prevalance...


([GradientBoostingClassifier(), 0.095762525968077, 0.2390998593530239],
 [GradientBoostingClassifier(), 0.18424584004294148, 0.6811594202898551])

In [68]:
training_stuff.all_quantifiers

[0.3348623853211009, 0.4969135802469136]

## Comparing with other algorithms

In [51]:
def groups_and_classes(ds):
    if ds == "compas":
        privileged_groups = [{"Z": 0}]
        unprivileged_groups = [{"Z": 1}]
        
    return {"p": privileged_groups, "up": unprivileged_groups}

## With reweighing
https://aif360.readthedocs.io/en/stable/modules/generated/aif360.algorithms.preprocessing.Reweighing.html

Faisal Kamiran and Toon Calders. 2012. Data preprocessing techniques for classification without discrimination. Knowledge and Information Systems 33, 1 (2012), 1–33.

In [59]:
from aif360.algorithms.preprocessing import Reweighing
from aif360.datasets import StandardDataset
from aif360.metrics import BinaryLabelDatasetMetric
from sklearn.preprocessing import StandardScaler

In [49]:
compas_dataset = StandardDataset(compas_df_2013, 
                          label_name='two_year_recid', 
                          favorable_classes=[0], 
                          protected_attribute_names=['Z'], 
                          privileged_classes=[[0]])

In [57]:

metric_orig_train = BinaryLabelDatasetMetric(compas_dataset, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_orig_train.mean_difference())

Difference in mean outcomes between unprivileged and privileged groups = -0.139718


In [52]:
privileged_groups = groups_and_classes('compas')["p"]
unprivileged_groups = groups_and_classes('compas')["up"]

In [54]:
RW = Reweighing(unprivileged_groups=unprivileged_groups,
               privileged_groups=privileged_groups)
RW.fit(compas_dataset)
compas_rw = RW.transform(compas_dataset)

In [58]:
metric_transf_train = BinaryLabelDatasetMetric(compas_rw, 
                                         unprivileged_groups=unprivileged_groups,
                                         privileged_groups=privileged_groups)
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_transf_train.mean_difference())

Difference in mean outcomes between unprivileged and privileged groups = 0.000000


In [63]:
scale_transf = StandardScaler()
X_train = scale_transf.fit_transform(compas_rw.features)
y_train = compas_rw.labels.ravel()

rwmod = LogisticRegression()
rwmod.fit(X_train, y_train,
        sample_weight=compas_rw.instance_weights)

LogisticRegression()

In [64]:
compas_dataset_test = StandardDataset(compas_df_2014, 
                          label_name='two_year_recid', 
                          favorable_classes=[0], 
                          protected_attribute_names=['Z'], 
                          privileged_classes=[[0]])

In [65]:
rwmod.score(X_test_2014, Y_test_2014)

/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


0.9415977961432507

## With meta-fr

In [69]:
from aif360.algorithms.inprocessing.meta_fair_classifier import MetaFairClassifier